In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install sentencepiece
!pip install transformers==4.21.2
!pip install tokenizers==0.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 182 kB 78.4 MB/s 
     |████████████████████████████████| 212 kB 79.4 MB/s 
     |████████████████████████████████| 115 kB 85.1 MB/s 
     |████████████████████████████████| 127 kB 68.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 4.8 MB/s 
     |████████████████████████████████| 6.6 MB 74.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels

In [3]:
!nvidia-smi

Sun Nov 27 16:42:32 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   28C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="microsoft/deberta-v3-base"
    # model='microsoft/deberta-base'  △
    # model='roberta-base'  
    # model='roberta-large'  x
    # model='roberta-large-mnli'
    # model='google/bigbird-roberta-base'
    # model='google/bigbird-roberta-large'
    # model='xlnet-large-cased'  △
    # model='albert-xxlarge-v2'
    # model="microsoft/deberta-large" o
    # model="microsoft/deberta-v3-large"  o
    # model='microsoft/deberta-v2-xlarge' x
    # model='microsoft/deberta-v2-xxlarge'
    # model='microsoft/deberta-xlarge' 
    # model='funnel-transformer/large' o
    # model='funnel-transformer/medium' △
    # model='albert-base-v2'
    # model='albert-large-v2'  x
    # model='google/electra-large-discriminator'  x
    # model='google/electra-base-discriminator'  x
    # model="facebook/bart-large-mnli"
    # model="facebook/bart-large"  o
    # model="facebook/bart-base"
    # model = "distilbert-base-uncased" x
    # model = "allenai/longformer-large-4096" x
    # model = "allenai/longformer-base-4096"
    # model = "uw-madison/yoso-4096"  x
    # model = "xlm-roberta-large"
    # model = "xlm-roberta-base"
    # model = "google/muril-large-cased" x
    # model = "google/rembert" x
    gradient_checkpointing=True
    scheduler='polynomial' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    use_prior_wd = True
    use_bertadam = False
    max_len=800
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_size=6
    fc_dropout=0.2
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    freezing=False
    mixout = 0.5
    is_mixout = True
    num_reinit_layers = 1
    is_reinit_layer = True
    fgm = True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [5]:
import os

DIR = '/content/drive/MyDrive/Competitions/Kaggle/FeedBack3'
INPUT_DIR = os.path.join(DIR,'input')
OUTPUT_DIR = os.path.join(DIR,'output')
OUTPUT_MODEL_DIR = DIR + '/output/EXP033/'
if not os.path.exists(OUTPUT_MODEL_DIR):
    os.makedirs(OUTPUT_MODEL_DIR)

In [6]:
# ====================================================
# Library
# ====================================================
from google.colab import runtime
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs


import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW, Optimizer
from torch.utils.data import DataLoader, Dataset
from torch.autograd.function import InplaceFunction 
import torch.nn.init as init

#os.system('pip uninstall -y transformers')
#os.system('pip uninstall -y tokenizers')
#os.system('python -m pip install --no-index --find-links=/content/drive/MyDrive/Competitions/Kaggle/FeedBack3/pip_wheel.ipynb transformers')
#os.system('python -m pip install --no-index --find-links=/content/drive/MyDrive/Competitions/Kaggle/FeedBack3/pip_wheel.ipynb tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


In [7]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_MODEL_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [8]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [9]:
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [10]:
def del_empty_text(text):
    """ Custom text cleaning. """
    text = text.strip()
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = re.sub(' +', ' ', text)
    text = text.strip()

    return text

In [11]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv(os.path.join(INPUT_DIR,'feedback3_train.csv'))

print(f"train shape: {train.shape}")
display(train.head(3))

train shape: (3911, 16)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold_a4,fold_a5,Topic,prob,Count,Bertopic,fold_c4,fold_c5
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,2,1,0,1.0,298,online classes home students,1,3
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0,0,44,1.0,20,change world want people,1,1
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,1,4,31,1.0,36,sports average policy play,3,4


In [12]:
train['full_text'] = train['full_text'].apply(resolve_encodings_and_normalize)
train['full_text'] = train['full_text'].apply(del_empty_text)

In [13]:
train['full_text'] = train['Bertopic'] + '[SEP]' + train['full_text']

display(train.head(3))

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold_a4,fold_a5,Topic,prob,Count,Bertopic,fold_c4,fold_c5
0,0016926B079C,online classes home students[SEP]I think that ...,3.5,3.5,3.0,3.0,4.0,3.0,2,1,0,1.0,298,online classes home students,1,3
1,0022683E9EA5,change world want people[SEP]When a problem is...,2.5,2.5,3.0,2.0,2.0,2.5,0,0,44,1.0,20,change world want people,1,1
2,00299B378633,"sports average policy play[SEP]Dear, Principal...",3.0,3.5,3.0,3.0,3.0,2.5,1,4,31,1.0,36,sports average policy play,3,4


In [14]:
display(train.groupby('fold_c5').size())

fold_c5
0    794
1    778
2    788
3    783
4    768
dtype: int64

In [15]:
if CFG.debug:
    display(train.groupby('fold_c5').size())
    train = train.sample(n=500, random_state=0).reset_index(drop=True)
    display(train.groupby('fold_c5').size())

if CFG.debug:
    pseudo_df = pseudo_df.sample(n=200, random_state=0).reset_index(drop=True)
    display(pseudo_df.groupby('fold_c5').size())

In [16]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
#tokenizer.save_pretrained(OUTPUT_MODEL_DIR+'/tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
class FGM():
    def __init__(self, model, eps=1.):
        self.model = model
        self.eps = eps
        self.backup = {}

    def attack(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = self.eps * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}

In [18]:
class Mixout(InplaceFunction):
    @staticmethod
    def _make_noise(input):
        return input.new().resize_as_(input)

    @classmethod
    def forward(cls, ctx, input, target=None, p=0.0, training=False, inplace=False):
        if p < 0 or p > 1:
            raise ValueError("A mix probability of mixout has to be between 0 and 1," " but got {}".format(p))
        if target is not None and input.size() != target.size():
            raise ValueError(
                "A target tensor size must match with a input tensor size {},"
                " but got {}".format(input.size(), target.size())
            )
        ctx.p = p
        ctx.training = training

        if ctx.p == 0 or not ctx.training:
            return input

        if target is None:
            target = cls._make_noise(input)
            target.fill_(0)
        target = target.to(input.device)

        if inplace:
            ctx.mark_dirty(input)
            output = input
        else:
            output = input.clone()

        ctx.noise = cls._make_noise(input)
        if len(ctx.noise.size()) == 1:
            ctx.noise.bernoulli_(1 - ctx.p)
        else:
            ctx.noise[0].bernoulli_(1 - ctx.p)
            ctx.noise = ctx.noise[0].repeat(input.size()[0], 1)
        ctx.noise.expand_as(input)

        if ctx.p == 1:
            output = target
        else:
            output = ((1 - ctx.noise) * target + ctx.noise * output - ctx.p * target) / (1 - ctx.p)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        if ctx.p > 0 and ctx.training:
            return grad_output * ctx.noise, None, None, None, None
        else:
            return grad_output, None, None, None, None


def mixout(input, target=None, p=0.0, training=False, inplace=False):
    return Mixout.apply(input, target, p, training, inplace)


class MixLinear(torch.nn.Module):
    __constants__ = ["bias", "in_features", "out_features"]
    def __init__(self, in_features, out_features, bias=True, target=None, p=0.0):
        super(MixLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter("bias", None)
        self.reset_parameters()
        self.target = target
        self.p = p

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return F.linear(input, mixout(self.weight, self.target, self.p, self.training), self.bias)

    def extra_repr(self):
        type = "drop" if self.target is None else "mix"
        return "{}={}, in_features={}, out_features={}, bias={}".format(
            type + "out", self.p, self.in_features, self.out_features, self.bias is not None
        )
    
def replace_mixout(model, mixout_p):
    for sup_module in model.modules():
        for name, module in sup_module.named_children():
            if isinstance(module, nn.Dropout):
                module.p = 0.0
            if isinstance(module, nn.Linear):
                target_state_dict = module.state_dict()
                bias = True if module.bias is not None else False
                new_module = MixLinear(
                    module.in_features, module.out_features, bias, target_state_dict["weight"], mixout_p
                )
                new_module.load_state_dict(target_state_dict)
                setattr(sup_module, name, new_module)
    return model

In [19]:
def reinit_layers(model):

    #for layer in model.model.encoder.layer[-CFG.num_reinit_layers:]:
    for layer in model.encoder.layer[-CFG.num_reinit_layers:]:    #Custome model内(backbone)

            for module in layer.modules():

                if isinstance(module,nn.Linear):
                    module.weight.data.normal_(mean=0.0,std=model.config.initializer_range)
                    if module.bias is not None:
                            module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)
                        
    return model

In [20]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.int32)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float32)
        return inputs, label
        
class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [21]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if cfg.is_mixout:
            self.model = replace_mixout(self.model, CFG.mixout)
            print(f"Initialized Mixout (p={CFG.mixout}) Regularization")
        if CFG.is_reinit_layer:
            self.model = reinit_layers(self.model)
            print(f'Reinitializing Last {CFG.num_reinit_layers} Layers.')
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
            
        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
            
        if 'microsoft/deberta-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:24].requires_grad_(False)
        if 'microsoft/deberta-v2-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:12].requires_grad_(False)
            

        self.high_dropout = nn.Dropout(p=0.5)
        
        self.fc = nn.Linear(self.config.hidden_size, cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)

        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        return feature, outputs
    

    def forward(self, inputs=None, labels=None):
        feature,outputs = self.feature(inputs)
        logits = torch.mean(
            torch.stack(
                [self.fc(self.high_dropout(feature)) for _ in range(5)],
                dim=0,
            ),
            dim=0,
        )

        # calculate loss
        loss = None
        if labels is not None:
            loss_fn = nn.SmoothL1Loss(reduction='mean')
            loss = loss_fn(logits, labels)
        
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [22]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

In [23]:
class PriorWD(Optimizer):
    def __init__(self, optim, use_prior_wd=False, exclude_last_group=True):
        super(PriorWD, self).__init__(optim.param_groups, optim.defaults)
        self.param_groups = optim.param_groups
        self.optim = optim
        self.use_prior_wd = use_prior_wd
        self.exclude_last_group = exclude_last_group
        self.weight_decay_by_group = []
        for i, group in enumerate(self.param_groups):
            self.weight_decay_by_group.append(group["weight_decay"])
            group["weight_decay"] = 0

        self.prior_params = {}
        for i, group in enumerate(self.param_groups):
            for p in group["params"]:
                self.prior_params[id(p)] = p.detach().clone()

    def step(self, closure=None):
        if self.use_prior_wd:
            for i, group in enumerate(self.param_groups):
                for p in group["params"]:
                    if self.exclude_last_group and i == len(self.param_groups):
                        p.data.add_(-group["lr"] * self.weight_decay_by_group[i], p.data)
                    else:
                        p.data.add_(
                            -group["lr"] * self.weight_decay_by_group[i], p.data - self.prior_params[id(p)],
                        )
        loss = self.optim.step(closure)

        return loss

    def compute_distance_to_prior(self, param):
        assert id(param) in self.prior_params, "parameter not in PriorWD optimizer"
        return (param.data - self.prior_params[id(param)]).pow(2).sum().sqrt()

In [24]:
def calculate_loss(inputs, labels, model, criterion, is_valid=True, device="cpu"):    
    outputs = model(inputs,labels)
    loss, logits = outputs[:2]
    return (loss, logits) if is_valid else loss

In [25]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, fgm):
    model.zero_grad()
    model.train()
    #awp_start = 1.0
    #optimizer.zero_grad()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        #if epoch >= awp_start:
        #    awp.perturb(inputs, labels, criterion)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
             loss = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=False, device=device)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        if scaler is not None:
            scaler.unscale_(optimizer)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        
        #awp.restore()
        if CFG.fgm:
          fgm.attack() 
          adversarial_loss = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=False, device=device)
          scaler.scale(adversarial_loss).backward()
          fgm.restore()
              
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  #'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          #grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
            
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            loss, y_preds = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=True, device=device)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(step, len(valid_loader),
                            loss=losses,
                            remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [26]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold_c5'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold_c5'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values
    
    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_MODEL_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr=5e-6, decoder_lr=1e-4, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        group1=['layer.0.','layer.1.','layer.2.','layer.3.']
        group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
        group3=['layer.8.','layer.9.','layer.10.','layer.11.']
        group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
        optimizer_parameters1 = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        optimizer_parameters2 = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': weight_decay, 'lr': encoder_lr/2.6},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': weight_decay, 'lr': encoder_lr},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': weight_decay, 'lr': encoder_lr*2.6},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.0},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.0, 'lr': encoder_lr/2.6},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.0, 'lr': encoder_lr},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.0, 'lr': encoder_lr*2.6},
        {'params': [p for n, p in model.named_parameters() if "model" not in n], 'lr':decoder_lr, "momentum" : 0.99},
    ]
        return optimizer_parameters2

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    optimizer = PriorWD(optimizer, use_prior_wd=CFG.use_prior_wd)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        elif cfg.scheduler == 'polynomial':
            warmup_steps = int(len(train_folds) / CFG.batch_size * 0.1)
            scheduler = get_polynomial_decay_schedule_with_warmup(
                optimizer, warmup_steps, num_train_steps, lr_end=7e-7, power=3.0)
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")

    print('Enable FGM')
    fgm = FGM(model=model, eps=0.1)

    best_score = np.inf

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, fgm)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            

    predictions = torch.load(OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [27]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_MODEL_DIR+'oof_df.pkl')

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

INFO:__main__:Deberta

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialized Mixout (p=0.5) Regularization
Reinitializing Last 1 Layers.
Enable FGM
Epoch: [1][0/389] Elapsed 0m 5s (remain 32m 36s) Loss: 2.9733(2.9733) LR: 0.00000053  
Epoch: [1][20/389] Elapsed 0m 32s (remain 9m 26s) Loss: 1.0013(1.8785) LR: 0.00001105  
Epoch: [1][40/389] Elapsed 0m 59s (remain 8m 26s) Loss: 0.2388(1.1431) LR: 0.00001989  
Epoch: [1][60/389] Elapsed 1m 26s (remain 7m 47s) Loss: 0.2235(0.8533) LR: 0.00001914  
Epoch: [1][80/389] Elapsed 1m 54s (remain 7m 14s) Loss: 0.1626(0.6879) LR: 0.00001841  
Epoch: [1][100/389] Elapsed 2m 21s (remain 6m 42s) Loss: 0.1681(0.5814) LR: 0.00001770  
Epoch: [1][120/389] Elapsed 2m 48s (remain 6m 12s) Loss: 0.1446(0.5151) LR: 0.00001701  
Epoch: [1][140/389] Elapsed 3m 16s (remain 5m 46s) Loss: 0.1736(0.4652) LR: 0.00001634  
Epoch: [1][160/389] Elapsed 3m 44s (remain 5m 17s) Loss: 0.1746(0.4281) LR: 0.00001568  
Epoch: [1][180/389] Elapsed 4m 11s (remain 4m 49s) Loss: 0.1530(0.3984) LR: 0.00001505  
Epoch: [1][200/389] Elapsed 4m 38

Epoch 1 - avg_train_loss: 0.2645  avg_val_loss: 0.1263  time: 556s
INFO:__main__:Epoch 1 - avg_train_loss: 0.2645  avg_val_loss: 0.1263  time: 556s
Epoch 1 - Score: 0.5053  Scores: [0.5406283784900634, 0.5086245388110906, 0.4706243324950617, 0.49508426337337724, 0.4969328744609669, 0.519745594530261]
INFO:__main__:Epoch 1 - Score: 0.5053  Scores: [0.5406283784900634, 0.5086245388110906, 0.4706243324950617, 0.49508426337337724, 0.4969328744609669, 0.519745594530261]
Epoch 1 - Save Best Score: 0.5053 Model
INFO:__main__:Epoch 1 - Save Best Score: 0.5053 Model


EVAL: [49/50] Elapsed 0m 17s (remain 0m 0s) Loss: 0.1036(0.1263) 
Epoch: [2][0/389] Elapsed 0m 1s (remain 10m 2s) Loss: 0.1736(0.1736) LR: 0.00000946  
Epoch: [2][20/389] Elapsed 0m 29s (remain 8m 44s) Loss: 0.1054(0.1414) LR: 0.00000901  
Epoch: [2][40/389] Elapsed 0m 57s (remain 8m 8s) Loss: 0.1554(0.1425) LR: 0.00000859  
Epoch: [2][60/389] Elapsed 1m 24s (remain 7m 34s) Loss: 0.1221(0.1516) LR: 0.00000818  
Epoch: [2][80/389] Elapsed 1m 52s (remain 7m 7s) Loss: 0.1205(0.1508) LR: 0.00000778  
Epoch: [2][100/389] Elapsed 2m 19s (remain 6m 38s) Loss: 0.2034(0.1502) LR: 0.00000739  
Epoch: [2][120/389] Elapsed 2m 47s (remain 6m 10s) Loss: 0.0914(0.1488) LR: 0.00000703  
Epoch: [2][140/389] Elapsed 3m 14s (remain 5m 41s) Loss: 0.0923(0.1473) LR: 0.00000667  
Epoch: [2][160/389] Elapsed 3m 41s (remain 5m 13s) Loss: 0.1215(0.1446) LR: 0.00000633  
Epoch: [2][180/389] Elapsed 4m 7s (remain 4m 44s) Loss: 0.1176(0.1426) LR: 0.00000600  
Epoch: [2][200/389] Elapsed 4m 34s (remain 4m 16s) Los

...

Epoch 3 - avg_train_loss: 0.1202  avg_val_loss: 0.1085  time: 551s
INFO:__main__:Epoch 3 - avg_train_loss: 0.1202  avg_val_loss: 0.1085  time: 551s
Epoch 3 - Score: 0.4664  Scores: [0.4937123623086892, 0.45492300438588223, 0.41104479858066195, 0.46787749750315333, 0.5059736704262252, 0.46494954189374854]
INFO:__main__:Epoch 3 - Score: 0.4664  Scores: [0.4937123623086892, 0.45492300438588223, 0.41104479858066195, 0.46787749750315333, 0.5059736704262252, 0.46494954189374854]
Epoch 3 - Save Best Score: 0.4664 Model
INFO:__main__:Epoch 3 - Save Best Score: 0.4664 Model


EVAL: [47/48] Elapsed 0m 17s (remain 0m 0s) Loss: 0.0860(0.1085) 
Epoch: [4][0/392] Elapsed 0m 1s (remain 11m 51s) Loss: 0.1226(0.1226) LR: 0.00000103  
Epoch: [4][20/392] Elapsed 0m 28s (remain 8m 29s) Loss: 0.0741(0.1279) LR: 0.00000098  
Epoch: [4][40/392] Elapsed 0m 55s (remain 7m 51s) Loss: 0.1470(0.1227) LR: 0.00000094  
Epoch: [4][60/392] Elapsed 1m 22s (remain 7m 25s) Loss: 0.1030(0.1189) LR: 0.00000090  
Epoch: [4][80/392] Elapsed 1m 49s (remain 6m 59s) Loss: 0.0757(0.1157) LR: 0.00000087  
Epoch: [4][100/392] Elapsed 2m 17s (remain 6m 34s) Loss: 0.0970(0.1194) LR: 0.00000084  
Epoch: [4][120/392] Elapsed 2m 45s (remain 6m 10s) Loss: 0.0869(0.1184) LR: 0.00000081  
Epoch: [4][140/392] Elapsed 3m 13s (remain 5m 44s) Loss: 0.1037(0.1180) LR: 0.00000079  
Epoch: [4][160/392] Elapsed 3m 41s (remain 5m 18s) Loss: 0.1505(0.1191) LR: 0.00000077  
Epoch: [4][180/392] Elapsed 4m 9s (remain 4m 50s) Loss: 0.0826(0.1182) LR: 0.00000075  
Epoch: [4][200/392] Elapsed 4m 35s (remain 4m 22s) 

Epoch 4 - avg_train_loss: 0.1173  avg_val_loss: 0.1040  time: 555s
INFO:__main__:Epoch 4 - avg_train_loss: 0.1173  avg_val_loss: 0.1040  time: 555s
Epoch 4 - Score: 0.4567  Scores: [0.48803448855241877, 0.44285332721100895, 0.40725127290143975, 0.4549283274246931, 0.4859603879703491, 0.46102797981358107]
INFO:__main__:Epoch 4 - Score: 0.4567  Scores: [0.48803448855241877, 0.44285332721100895, 0.40725127290143975, 0.4549283274246931, 0.4859603879703491, 0.46102797981358107]
Epoch 4 - Save Best Score: 0.4567 Model
INFO:__main__:Epoch 4 - Save Best Score: 0.4567 Model


EVAL: [47/48] Elapsed 0m 17s (remain 0m 0s) Loss: 0.0842(0.1040) 


========== fold: 4 result ==========
INFO:__main__:========== fold: 4 result ==========
Score: 0.4567  Scores: [0.48803448855241877, 0.44285332721100895, 0.40725127290143975, 0.4549283274246931, 0.4859603879703491, 0.46102797981358107]
INFO:__main__:Score: 0.4567  Scores: [0.48803448855241877, 0.44285332721100895, 0.40725127290143975, 0.4549283274246931, 0.4859603879703491, 0.46102797981358107]
========== CV ==========
INFO:__main__:========== CV ==========
Score: 0.4578  Scores: [0.49274793221814706, 0.4496425972904012, 0.419665405708347, 0.4568491878101974, 0.4760050014204179, 0.45201888473307594]
INFO:__main__:Score: 0.4578  Scores: [0.49274793221814706, 0.4496425972904012, 0.419665405708347, 0.4568491878101974, 0.4760050014204179, 0.45201888473307594]


In [ ]:
runtime.unassign()